# Next steps

In this section, we will look at some topics which are perhaps not necessary for writing a module but which are definitively worth considering.

## Testing your code

To show that your code works as expected and to ensure that it will continue to work in future, you should write a test for your new module. Such test can be written as a general script in Python or Bash. There is also the Python [doctest](https://docs.python.org/2/library/doctest.html) package to test small portions of code in the module. However, in the latest GRASS GIS, there is a dedicated unit testing Python package [gunittest](http://grass.osgeo.org/grass71/manuals/libpython/gunittest_testing.html) which contains functions to make testing of GRASS GIS modules as convenient as possible. The `grass.gunittest` package is currently available in 'trunk' (the [development version](http://grass.osgeo.org/download/software/sources/), currently version 7.1).

The tests are written as a function (method) of a class which inherits from `TestCase` available in `grass.gunittest.case` package. Test function name must start with `test_`. There can be one or more of these `test_` functions. The `runModule()` function is used to call modules we are not testing directly. We use `assertModule()` function to call the module we are testing with given parameters, and we expect the process to finish successfully. Functions `setUp()` and `tearDown()` are called before and after every call of the `test_` functions. We should always delete the created maps (using `g.remove`).


In [ ]:
%%python
# Only available for GRASS GIS 7.1 (and above)
from grass.gunittest.case import TestCase
from grass.gunittest.main import test
import grass.script as gscript

class TestViewshedPoint(TestCase):

    prefix = 'test_rviewshedpoints_'

    @classmethod
    def setUpClass(cls):
        """Ensure expected computational region and create input vector points"""
        cls.use_temp_region()
        cls.runModule('g.region', n=225200, s=222500, w=637500, e=640000, raster='elevation')
        cls.runModule('v.random', output=cls.prefix + 'input_points', npoints=10, seed=2)

    def setUp(self):
        """Create r.viewshed.points output for every test method"""
        self.runModule('r.viewshed.points', elevation='elevation',
                       points=self.prefix + 'input_points', output_points=self.prefix + 'output_points',
                       viewshed_basename=self.prefix + 'output_viewshed')

    def tearDown(self):
        """Remove r.viewshed.points output after every test"""
        self.runModule('g.remove', type=['raster', 'vector'], pattern=self.prefix + '*', flags='f')

    def test_outputs_exists(self):
        """Test that output maps exist"""
        num_viewsheds = len(gscript.list_grouped(['raster'],
                                                 pattern=self.prefix + '*')[gscript.gisenv()['MAPSET']])
        self.assertEqual(first=10, second=num_viewsheds, msg="Wrong number of viewsheds computed")
        self.assertVectorExists(self.prefix + 'output_points')
        
    def test_area_matches(self):
        """Test if areas are correct"""
        minmax = {'min': 392800, 'max': 1759000}
        self.assertVectorFitsUnivar(map=self.prefix + 'output_points', column='area',
                                    reference=minmax, precision=1e-6)


if __name__ == '__main__':
    test()


## Creating a GUI toolbox

You may be interested to get your new Python module into the standard graphical user interface of GRASS GIS. GUI toolboxes are a way to customize items in wxGUI menu. You just need to generate a XML file and copy it to a predefined directory. The XML file may look like this:

In [ ]:
%%file toolboxes.xml
<?xml version="1.0" encoding="UTF-8"?>
<toolboxes>
  <toolbox name="Viewsheds">
    <label>Viewsheds</label>
    <items>
      <module-item name="r.viewshed.points">
        <label>Compute special viewsheds</label>
      </module-item>
      <module-item name="r.viewshed">
        <label>Compute standard viewshed</label>
      </module-item>
      <module-item name="g.region">
        <label>Set region</label>
      </module-item>
      <module-item name="v.random">
        <label>Generate random points</label>
      </module-item>
      <separator/>
    </items>
  </toolbox>
</toolboxes>

You have to copy this file `toolboxes.xml` to the GRASS GIS configuration directory in your home directory (e.g., `~/.grass7/toolboxes/toolboxes.xml` on Linux). The next time you start the GRASS GIS GUI (`g.gui`), you will see the new toolbox under 'user defined toolboxes' in the module tree in the *Search module* tab in the *Layer manager* window.

## GRASS GIS community

In case of questions, the best place to ask is on the [grass-user](http://lists.osgeo.org/listinfo/grass-user) mailing list. This is also a great place to announce your new addon.

If you are developing a new addon and you need help from developers you can write do the [grass-dev](http://lists.osgeo.org/listinfo/grass-dev) mailing list.

The are also some localized GRASS GIS mailing list, you can find them on the [dedicated mailing list web page](http://grass.osgeo.org/support/mailing-lists/).

There is also a GRASS GIS community on [Google+](https://plus.google.com/communities/111147786674687562495) waiting for your GRASS GIS related news and a [Facebook group](https://www.facebook.com/groups/96121775724/).


## What's next?

We are looking for people with all kind of skills to improve the GRASS GIS project: we are looking for translators, web designers, web content manager, programmers, teaching and curriculum people, and GRASS GIS advocates.

You can also help us [sponsoring](http://grass.osgeo.org/support/our-sponsors/) GRASS GIS (also small donations are welcome!) or you may [report errors or enhancement wishes](http://grass.osgeo.org/development/bug-tracking/). Get involved!